# sound API


In [1]:
import pyaudio
import wave
import time
import sys
import numpy as np

### sound playing function :

In [2]:
def number_reader(number_string):
    file_name = "audio-numbers\\" + number_string + ".wav"
    print(file_name)
    wf = wave.open(file_name, 'rb')
    p = pyaudio.PyAudio()

    def callback(in_data, frame_count, time_info, status):
        data = wf.readframes(frame_count)
        return (data, pyaudio.paContinue)

    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True,
                    stream_callback=callback)

    stream.start_stream()

    while stream.is_active():
        time.sleep(0.1)

    stream.stop_stream()
    stream.close()
    wf.close()

    p.terminate()
    return None

### number division function:

In [3]:
def number_reader_api(number):
    if number < 100 :
        number_reader(str(number))
        return None
    elif ((number >= 100) and (number < 1000)):
        hund = int(100 * np.floor(number/100))
        rest =  int(number - hund )
        number_reader(str(hund))
        number_reader("and")
        number_reader(str(rest))  
        return None
    elif(number >= 1000) and (number < 10000):
        thousand = int(1000 *  np.floor(number/1000))
        rest1 = int( (number - thousand ))
        hund =int( 100 *  np.floor(rest1/100))
        rest = int( (rest1 - hund ))
        number_reader(str(thousand))
        number_reader("and")
        number_reader(str(hund))
        number_reader("and")
        number_reader(str(rest))  
        return None
    else :
        number_reader(str(10000))
        return None
    return None

### how to use it:

In [4]:
number_reader_api(5555)

audio-numbers\5000.wav
audio-numbers\and.wav
audio-numbers\500.wav
audio-numbers\and.wav
audio-numbers\55.wav


## image processing:

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
class our_templates:
    templates_values = []
    num_of_templates = 0 
    templates_list  =[]
    #add any new templates here
    def __init__(self):
        
        half_pound_front=cv2.imread('dataset//templates//half_pound_coin_front.png',0)
        self.templates_list.append(half_pound_front)
        self.templates_values.append(0.5)
        
        half_pound_back=cv2.imread('dataset//templates//half_pound_coin_back.png',0)
        self.templates_list.append(half_pound_back)
        self.templates_values.append(0.5)
        
        one_pound_back=cv2.imread('dataset//templates//one_pound_coin_back.png',0)
        self.templates_list.append(one_pound_back)
        self.templates_values.append(1)
        
        one_pound_front=cv2.imread('dataset//templates//one_pound_coin_front.png',0)
        self.templates_list.append(one_pound_front)
        self.templates_values.append(1)

        one_pound_back_paper=cv2.imread('dataset//templates//one_pound_paper_back.png',0)
        self.templates_list.append(one_pound_back_paper)
        self.templates_values.append(1)
        
        one_pound_front_paper=cv2.imread('dataset//templates//one_pound_paper_front.png',0)
        self.templates_list.append(one_pound_front_paper)
        self.templates_values.append(1)      
        
        five_pound_front_paper=cv2.imread('dataset//templates//5 part_1.jpg',0)
        self.templates_list.append(five_pound_front_paper)
        self.templates_values.append(5)
        
        five_pound_back_paper=cv2.imread('dataset//templates//five_pound_paper_back.jpg',0)
        self.templates_list.append(five_pound_back_paper)
        self.templates_values.append(5)
        
                
        ten_pound_front=cv2.imread('dataset//templates//ten_pound_front.jpg',0)
        self.templates_list.append(ten_pound_front)
        self.templates_values.append(10)
        
                        
        ten_pound_back=cv2.imread('dataset//templates//ten_pound_back.png',0)
        self.templates_list.append(ten_pound_back)
        self.templates_values.append(10)
        
        twenty_pound_front=cv2.imread('dataset//templates//20 front.jpg',0)
        self.templates_list.append(twenty_pound_front)
        self.templates_values.append(20)
        
                        
        twenty_pound_back=cv2.imread('dataset//templates//20 back.jpg',0)
        self.templates_list.append(twenty_pound_back)
        self.templates_values.append(20)
        
        fifty_pound_front=cv2.imread('dataset//templates//50 front.jpg',0)
        self.templates_list.append(twenty_pound_front)
        self.templates_values.append(50)
        
                        
        fifty_pound_back=cv2.imread('dataset//templates//50_pound_back.jpg',0)
        self.templates_list.append(twenty_pound_back)
        self.templates_values.append(50)
        
        hundred_pound_front=cv2.imread('dataset//templates//hundred_front.jpg',0)
        self.templates_list.append(hundred_pound_front)
        self.templates_values.append(100)

        hundred_pound_back=cv2.imread('dataset//templates//hundred_back.jpg',0)
        self.templates_list.append(hundred_pound_back)
        self.templates_values.append(100)
        
        two_hundred_pound_front=cv2.imread('dataset//templates//200 front.jpg',0)
        self.templates_list.append(two_hundred_pound_front)
        self.templates_values.append(200)

        two_hundred_pound_back=cv2.imread('dataset//templates//200 back.jpg',0)
        self.templates_list.append(two_hundred_pound_back)
        self.templates_values.append(200)
        
        
        self.num_of_templates = len(self.templates_values)
        
        
        

In [3]:
def preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,127,255,0)
    _,contours,_ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    c = max(contours, key = cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    #crop the banknote
    img_2=img[y:y+h,x:x+w]
    img_2=cv2.cvtColor(img_2, cv2.COLOR_BGR2GRAY)
    #apply adaptive histogram equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(img_2)
    
    
    return cl1
    
    

In [4]:
## mobile camera test
samples_list = []
#five samples

five1 =cv2.imread('dataset//five_samples//five1.png')
five1=preprocess(five1)
samples_list.append(five1)

five2 =cv2.imread('dataset//five_samples//five2.png')
five2=preprocess(five2)
samples_list.append(five2)


five3 =cv2.imread('dataset//five_samples//five3.png')
five3=preprocess(five3)
samples_list.append(five3)

five4 =cv2.imread('dataset//five_samples//five4.png')
five4=preprocess(five4)
samples_list.append(five4)


ten_1=cv2.imread('dataset//ten_samples//ten1.jpg')
ten_1=preprocess(ten_1)
samples_list.append(ten_1)


ten_2=cv2.imread('dataset//ten_samples//ten2.jpg')
ten_2=preprocess(ten_2)
samples_list.append(ten_2)


ten_3=cv2.imread('dataset//ten_samples//ten3.jpg')
ten_3=preprocess(ten_3)
samples_list.append(ten_3)


ten_4=cv2.imread('dataset//ten_samples//ten4.png')
ten_4=preprocess(ten_4)
samples_list.append(ten_4)

ten_5 =cv2.imread('images//ten_pound_front.jpg')
ten_5=preprocess(ten_5)
samples_list.append(ten_5)


#hundred_1 =cv2.imread('dataset//hundred samples//front_1.png')
#samples_list.append(hundred_1)


#hundred_2 =cv2.imread('dataset//hundred samples//front_2.jpg')
#samples_list.append(hundred_2)

#hundred_3 =cv2.imread('dataset//hundred samples//back_1.jpg')
#samples_list.append(hundred_3)


#hundred_4 =cv2.imread('dataset//hundred samples//back_2.jpg')
#samples_list.append(hundred_4)



num_of_samples = len(samples_list)

In [5]:
"""
this function is applied on each template , to get its featues ,, 
then we could match it with any input image that will be tested
"""
def get_currency_feature(img):
    #to ensure that image is in gray scale
   # img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #declaring orb object
    orb=cv2.ORB_create()
    #find key points with ORB
    kp = orb.detect(img,None)
    # compute the descriptors with ORB
    kp, des = orb.compute(img, kp)
    
    return kp, des
    

In [6]:
keys_and_des = []

for i in range(0,num_of_samples):
    keys_and_des.append( get_currency_feature(samples_list[i]) )

In [7]:
templatess = our_templates()

____________________________________________________________________________________________________________________

In [8]:
def guess(template_image, input_Kp, input_des): #kp,des of the input image , x is the images/
    template_kp,template_des=get_currency_feature(template_image)
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    # Match descriptors.
    matches = bf.match(input_des, template_des)
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    sum_of_matches=0 
    
    n_of_matches = 10
    length=len(matches)
    print ("length",length)
    for i in range (0 , n_of_matches):
        sum_of_matches = sum_of_matches+ matches[i].distance  # getting list of top 10 matches' distances
    return  sum_of_matches


In [9]:

def estimate( input_kp, input_des , templates):
    overall_matches=[] 
    for i in range(0 ,18):  #templates.num_of_templates
        overall_matches.append( guess(templates.templates_list[i] ,input_kp , input_des )  )
    minimum_dis = overall_matches.index(min(overall_matches)) #getting index of the predicted banknote
    
    predicted = templates.templates_values[minimum_dis]
#    print (overall_matches)
    print("preddicted = {0} ,index = {1}".format(predicted , minimum_dis))
    print ("________________________________________________________________________________")
    return predicted


In [10]:
results = []
for i in range(0,num_of_samples):
    estimate(keys_and_des[i][0] , keys_and_des[i][1] , templatess)
templatess.num_of_samples=18

('length', 255)
('length', 252)
('length', 272)
('length', 249)
('length', 181)
('length', 152)
('length', 222)
('length', 184)
('length', 216)
('length', 241)
('length', 229)
('length', 237)
('length', 229)
('length', 237)
('length', 212)
('length', 240)
('length', 222)
('length', 248)
preddicted = 5 ,index = 6
________________________________________________________________________________
('length', 263)
('length', 251)
('length', 272)
('length', 244)
('length', 173)
('length', 144)
('length', 234)
('length', 180)
('length', 211)
('length', 239)
('length', 235)
('length', 243)
('length', 235)
('length', 243)
('length', 208)
('length', 241)
('length', 219)
('length', 252)
preddicted = 5 ,index = 6
________________________________________________________________________________
('length', 269)
('length', 256)
('length', 276)
('length', 255)
('length', 184)
('length', 146)
('length', 221)
('length', 179)
('length', 204)
('length', 242)
('length', 235)
('length', 239)
('length', 235)
('